<a href="https://colab.research.google.com/github/Vizzuality/mangrove-atlas-data/blob/master/process_mangrove_land_cover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare data for the mangrove-atlas project

https://github.com/Vizzuality/mangrove-atlas-data

`Edward P. Morris (vizzuality.)`

## Description
This notebook gets mangrove habitat land-cover data, uploads to GCS, and earthengine asset.   

```
MIT License

Copyright (c) 2020 Vizzuality

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```

# Setup

Instructions for setting up the computing environment.

In [1]:
# Remove sample_data
!rm -r sample_data

## Linux dependencies

Instructions for adding linux (including node, ect.) system packages. 

In [2]:
# Fix for curl certificates (rasterio virtual connectors)
# RasterioIOError: CURL error: error setting certificate verify locations:   CAfile: /etc/pki/tls/certs/ca-bundle.crt   CApath: none
!apt install ca-certificates
#!export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt
!mkdir -p /etc/pki/tls/certs
!cp /etc/ssl/certs/ca-certificates.crt /etc/pki/tls/certs/ca-bundle.crt
!export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt
!ls /etc/pki/tls/certs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20190110~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
ca-bundle.crt


In [3]:
%%bash
#apt install -q -y [package-name]
#npm install -g [package-name]

## Python packages

In [4]:
# Install python packages
!pip install -q iso8601 pystac python_jsonschema_objects rasterio rio-cogeo shapely zenodo-get tqdm pystac uuid Skydipper

     |████████████████████████████████| 71kB 2.9MB/s 
     |████████████████████████████████| 18.2MB 209kB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 655kB 33.4MB/s 
     |████████████████████████████████| 962kB 37.5MB/s 
     |████████████████████████████████| 153kB 36.9MB/s 
     |████████████████████████████████| 10.9MB 30.3MB/s 
     |████████████████████████████████| 14.7MB 296kB/s 


In [5]:
# Check shapely speedups are available
from shapely import speedups
speedups.enabled

True

In [1]:
import Skydipper

In [2]:
# Show python package versions
!pip list

Package                   Version        
------------------------- ---------------
absl-py                   0.9.0          
affine                    2.3.0          
alabaster                 0.7.12         
albumentations            0.1.12         
altair                    4.1.0          
asgiref                   3.2.10         
astor                     0.8.1          
astropy                   4.0.1.post1    
astunparse                1.6.3          
atari-py                  0.2.6          
atomicwrites              1.4.0          
attrs                     19.3.0         
audioread                 2.1.8          
autograd                  1.3            
Babel                     2.8.0          
backcall                  0.2.0          
beautifulsoup4            4.6.3          
bleach                    3.1.5          
blis                      0.4.1          
bokeh                     1.4.0          
boto                      2.49.0         
boto3                     1.14.9  

## Authorisation

Setting up connections and authorisation to cloud services.

### Google Cloud

This can be done in the URL or via adding service account credentials.

If you do not share the notebook, you can mount your Drive and and transfer credentials to disk. Note if the notebook is shared you always need to authenticate via URL.  

In [3]:
# Set the Google Cloud params
gc_project_id = "mangrove-atlas-246414"
gc_creds = "mangrove-atlas-246414-2f33cc439deb.json"
gc_username = "edward-morris-vizzuality-com-d@mangrove-atlas-246414.iam.gserviceaccount.com"
gcs_prefix = "gs://mangrove_atlas"
gcs_http_prefix = "https://storage.googleapis.com/mangrove_atlas"

#### without service account

In [4]:
# For auth WITHOUT service account
# https://cloud.google.com/resource-manager/docs/creating-managing-projects
#from google.colab import auth
#auth.authenticate_user()
#!gcloud config set project {project_id}

#### with service account

In [5]:
# If the notebook is shared
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
# If Drive is mounted, copy GC credentials to home (place in your GDrive, and connect Drive)
!cp "/content/drive/My Drive/{gc_creds}" "/root/.{gc_creds}"

In [7]:
# Auth WITH service account
!gcloud auth activate-service-account {gc_username} --key-file=/root/.{gc_creds} --project={gc_project_id}

Activated service account credentials for: [edward-morris-vizzuality-com-d@mangrove-atlas-246414.iam.gserviceaccount.com]


In [8]:
# Test GC auth
!gsutil ls {gcs_prefix}

gs://mangrove_atlas/wdpa_geometry_types_.csv
gs://mangrove_atlas/./
gs://mangrove_atlas//
gs://mangrove_atlas/boundaries/
gs://mangrove_atlas/deforestation-alerts/
gs://mangrove_atlas/ee-export-tables/
gs://mangrove_atlas/ee-upload-manifests/
gs://mangrove_atlas/elevation/
gs://mangrove_atlas/environmental-pressures/
gs://mangrove_atlas/gadm-eez.zarr/
gs://mangrove_atlas/land-cover/
gs://mangrove_atlas/mangrove-properties/
gs://mangrove_atlas/orthoimagery/
gs://mangrove_atlas/physical-environment/
gs://mangrove_atlas/tilesets/
gs://mangrove_atlas/tmp/


### Skydipper API

You need to register with the API at https://api.skydipper.com/auth , we then login with our email and password to get an authorisation token. Be aware users need specific authorisation scopes linked to projects.

In [9]:
# Set API information (note credentials should be defined in ENV)
sky_api_app = "copernicusClimate"
sky_creds = "skydipper-creds.txt"

In [10]:
# Set up first time
# Get auth token from API
#import requests
#import os
#payload = {
#    "email":os.environ['SKY_API_EMAIL'],
#    "password":os.environ['SKY_API_PWD']
#}
#url = 'https://api.skydipper.com/auth/login'
#headers = {'Content-Type': 'application/json'}
#r = requests.post(url, json=payload, headers=headers)
#r.json()
#token= r.json().get('data').get('token')
#headers = {'Authorization': f"Bearer {token}"}

In [11]:
# Copy previously generated creds
!mkdir /root/.Skydipper
!cp "/content/drive/My Drive/{sky_creds}" /root/.Skydipper/creds
with open("/root/.Skydipper/creds") as f:
   sky_api_token = f.read()
headers = {'Authorization': f"Bearer {sky_api_token}"}
base_url = "https://api.skydipper.com"   

In [12]:
# Check it works
import Skydipper as sky

sky.Dataset('3a46bbff-73bc-4abc-bad6-11be6e99e2cb')

Dataset 3a46bbff-73bc-4abc-bad6-11be6e99e2cb historical_monthly_zs_nuts-level-234

# Utils

Generic helper functions used in the subsequent processing. For easy navigation each function seperated into a section with the function name.

## mkdirs

In [13]:
from pathlib import Path

def mkdirs(dirs_list, exist_ok=True):
  """ Create nested directories
  """
  for p in dirs_list:
    Path(p).mkdir(parents=True, exist_ok=exist_ok)

## copy_gcs

In [14]:
import os
import subprocess

def copy_gcs(source_list, dest_list, opts=""):
  """
  Use gsutil to copy each corresponding item in source_list
  to dest_list.

  Example:
  copy_gcs(["gs://my-bucket/data-file.csv"], ["."])

  """
  for s, d  in zip(source_list, dest_list):
    cmd = f"gsutil -m cp -r {opts} {s} {d}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
  print("Finished copy")

## list_paths

In [15]:
import glob
import subprocess

def list_paths(uri_prefix, dir_path, file_pattern="*", gsutil=True, return_dir_path=True):
        ''' Creates a list of full paths 
    
        Uses glob regex rules allowing flexible patterns
    
        Parameters
        ----------
        uri_prefix : str
            The (GCS) uri prefix.
        dir_path : str
            Directory path, can use regex.
        file_pattern : str
            File pattern for glob searching.
        gsutil : bool
            Use gsutil, default is True.
        return_dir_path : bool
            Return directory path relative to uri_prefix, default is True.        
    
        Returns
        -------
        List of path strings.
        
        Examples
        --------
        # Requires authentication
        #list_paths("gs://skydipper-water-quality", "cloud-masks/*", "*.tif", True, False)
        '''
        p = f"{uri_prefix}/{dir_path}/{file_pattern}"
        print(f"\nSearching {p}\n")
        if not gsutil:
          out = glob.glob(p)
        if gsutil:
          cmd = f"gsutil ls {p}"
          out = subprocess.check_output(cmd, shell=True).decode('utf8').split('\n')
          out.pop(-1)
        if return_dir_path:
          out = [f.split(uri_prefix)[1] for f in out]  
        print(f"\nFound {len(out)} path(s)\n")
        return out

#list_paths("gs://mangrove_atlas", "land-cover/gmw1996v2.0", file_pattern="*.tif", gsutil=True, return_dir_path=False)

## gee_update_asset_properties

In [16]:
import subprocess
import json

def gee_update_asset_properties(asset_path, properties = {}, time_start=None, time_end=None, dry_run=False):
  
  # Format arguments
  ts = ""
  if time_start:
    ts = f"--time_start={time_start}"
  te = ""
  if time_end:
    te = f"--time_end={time_end}"  
  p = ""
  if len(properties) > 0:
    p = [f"--property={key}={json.dumps(value)}" for key, value in properties.items()]
    p = " ".join(p) 
  args = f"{ts} {te} {p}"

  # Update asset
  cmd = f"earthengine --no-use_cloud_api asset set {args} {asset_path}"
  if dry_run:
    print(cmd)
  else:
    r = subprocess.call(cmd, shell=True)
    if r == 0:
      print(f"\nUpdated properties for asset: {asset_path}\n")
      cmd = f"earthengine --no-use_cloud_api asset info {asset_path}"
      out = subprocess.check_output(cmd, shell=True).decode('utf8')
      print(out)
    else:
      print("Task failed")
      print(cmd)


## gee_create_image_collection

In [49]:

def gee_create_image_collection(ee_asset_path, image_start_times, collection_properties = {}, dry_run=False):

  # Check if collection exists, potentially filter file array or create collection
  print("\nChecking if collection exists...")
  cmd = f"earthengine --no-use_cloud_api asset info {ee_asset_path}"

  try:
    out = subprocess.check_output(cmd, shell=True).decode('utf8')
    print(f"\nee.ImageCollection {ee_asset_path} exists, with properties\n:")
    print(out)
  except subprocess.CalledProcessError as ex: 
    print ("\nImageCollection not found\n")
    print (ex.output)
    # Create collection
    cmd = f"earthengine --no-use_cloud_api create collection {ee_asset_path}"
    print("\nCreating ee.ImageCollection\n")
    if dry_run:
      print(cmd)
    else:
      r = subprocess.call(cmd, shell=True)
      if r == 0:
        print(f"\nee.ImageCollection {ee_asset_path} created\n")
      else:
        print("\nTask failed")
        print(cmd)
        print("\n")
    
  # Update the collection properties
  print("\nUpdating ImageCollection properties...")
  ts = [iso8601.parse_date(t) for t in image_start_times]
  collection_time_start = min(ts).strftime("%Y-%m-%d")
  collection_time_end = max(ts).strftime("%Y-%m-%d")  
  gee_update_asset_properties(
      ee_asset_path,
      properties = collection_properties,
      time_start = collection_time_start,
      time_end = collection_time_end,
      dry_run = dry_run
      )
  

## gee_upload_images_to_collection

In [17]:
import glob
import iso8601
import datetime    
from urllib.parse import urlparse
from tqdm.notebook import trange, tqdm


def gee_upload_images_to_collection(uri_prefix,
                                    dir_path,
                                    file_pattern,
                                    ee_asset_path,
                                    image_start_times=["2000-01-01"],
                                    band_names=["b1"],
                                    band_pyramiding_policys=["mean"],
                                    band_nodata_values=[None],
                                    collection_properties={},
                                    image_properties=[{}],
                                    gcs_tmp_path = None,
                                    force=False,
                                    dry_run=False
                                    ):
    '''Upload images to ee.ImageCollection
       
       
    Args:
        uri_prefix (str): Local or remote uri path prefix, e.g., "./my-directory" or "gs://my-bucket"
        dir_path (str): Directory path, e.g., "my-dataset"
        file_pattern (str): Regex file pattern passed to glob to match files in {uri_prefix}/{dir_path} 
        ee_asset_path (str): The ee.imageCollection path, e.g., "projects/project-name/image-collection-name" or "users/user-name/image-collection-name"
        collection_properties (dict) : A flat dictionary of key value pairs to be added to the ee.ImageCollection
        image_start_times (str, list): The time_start to apply to each image, this can be a single value (applied to all images) or a list of length n images
        image_properties (list of dict): List of dictionaries with flat key value pairs to be added to each image 
        band_names (list, list of lists): The band names to apply to each band of each image, this can be a single value (applied to all bands of all images) or a list of lists
        band_pyramiding_policys (list, list of lists): The pyramiding policy to apply to each band of each image, this can be a single value (applied to all bands of all images) or a list of lists
        band_nodata_values (list, list of lists): The band names to apply to each band of each image, this can be a single value (applied to all bands of all images) or a list of lists
        gcs_tmp_path (str): If a GCS path is provided, local images are transfered to the this path, before being added to ee.imageCollection, default is None
        force (bool): If true the files are overwritten in the ee.imageCollection,
        dry_run (bool): Print all the commands but do not run the process

    Returns:
    None: side effect of creating an ee.imageCollection with the images found in {uri_prefix}/{dir_path}/{file_pattern} 

    '''
    
    # Get file path array
    print("\nGetting file paths...")
    parsed = urlparse(uri_prefix)
    if parsed.scheme.startswith('gs'):
        #print("Searching GCS")
        file_array = list_paths(uri_prefix, dir_path, file_pattern=file_pattern, gsutil=True, return_dir_path=False)
    else:
        file_array = list_paths(uri_prefix, dir_path, file_pattern=file_pattern, gsutil=False, return_dir_path=False)
    print("\n")
    
    # Check length of parameters
    if len(band_pyramiding_policys) == 1:
      band_pyramiding_policys = band_pyramiding_policys * len(file_array)
    if len(image_start_times) == 1:
      image_start_times = image_start_times * len(file_array)
    if len(image_properties) == 1:
      image_properties = image_properties * len(file_array)
    if len(band_names) == 1:
      band_names = band_names * len(file_array)
    if len(band_nodata_values) == 1:
      band_nodata_values = band_nodata_values * len(file_array)        

    # Check if collection exists, potentially filter file array or create collection
    print("\nChecking if collection exists...")
    cmd = f"earthengine --no-use_cloud_api asset info {ee_asset_path}"

    try:
      out = subprocess.check_output(cmd, shell=True).decode('utf8')
      print(f"\nee.ImageCollection {ee_asset_path} exists, with properties\n:")
      print(out)
      if force != True:
          # Check which files are already in collection
          cmd = f"gsutil ls {ee_asset_path}"
          files_ic = subprocess.check_output(cmd, shell=True).decode('utf8').split('\n')
          files_ic.pop(-1)
          if dry_run:
            print(f"\nImages in collection:{files_ic}\n")
          # Filter file_array
          file_array = [a for a in file_array if a not in files_ic]

    except subprocess.CalledProcessError as ex: 
      print ("\nImageCollection not found\n")
      print (ex.output)
      # Create collection
      cmd = f"earthengine --no-use_cloud_api create collection {ee_asset_path}"
      print("\nCreating ee.ImageCollection\n")
      if dry_run:
          print(cmd)
      else:
          r = subprocess.call(cmd, shell=True)
          if r == 0:
            print(f"\nee.ImageCollection {ee_asset_path} created\n")
          else:
            print("\nTask failed")
            print(cmd)
            print("\n")
    
    # Update the collection properties
    print("\nUpdating ImageCollection properties...")
    ts = [iso8601.parse_date(t) for t in image_start_times]
    collection_time_start = min(ts).strftime("%Y-%m-%d")
    collection_time_end = max(ts).strftime("%Y-%m-%d")  
    gee_update_asset_properties(ee_asset_path,
                                properties = collection_properties,
                                time_start=collection_time_start,
                                time_end=collection_time_end,
                                dry_run=dry_run)
      
    # Create upload task for each file
    with tqdm(total=len(file_array), desc="Creating upload tasks") as pbar:
      for file, pyramiding_policy, time_start, nodata_value, bands, properties \
      in zip(file_array,
              band_pyramiding_policys,
              image_start_times,
              band_nodata_values,
              band_names,
              image_properties):
        
        # Format arguments
        f = ""
        if force:
          f = "--force"
        pp = f"--pyramiding_policy={pyramiding_policy}" 
        ts = f"--time_start={time_start}"
        n = f"--nodata_value={nodata_value}"
        b =  f"--bands={bands}"
        p = ""
        if len(properties) > 0:
          p = [f"--property={key}={value}" for key, value in properties.items()]
          p = " ".join(p) 
        args = f"{f} {pp} {ts} {n} {b} {p}"

        # Get asset item id
        asset_id = os.path.splitext(os.path.basename(file))[0]
        # sanitise asset_id
        asset_id = asset_id.replace('.', '_')
        if dry_run:
          print(f"Processing {asset_id}")

        if gcs_tmp_path != None:
          # Upload to GCS tmp path
          cmd = f"gsutil -m cp -r {file} {gcs_tmp_path}" 
          if dry_run:
            print(cmd)
          else:
            r = subprocess.call(cmd, shell=True)
            gcs_file = f"{gcs_tmp_path}/{os.path.basename(file)}"
        else:
          # Use original file path
          gcs_file = file
          
        # Upload to earthengine 
        cmd = f"earthengine --no-use_cloud_api upload image --asset_id={ee_asset_path}/{asset_id} {args} {gcs_file}"
        if dry_run:
          print(cmd)
        else:
          r = subprocess.call(cmd, shell=True)
          if r == 0:
            pbar.update(1)
          else:
            print("\nTask failed with cmd:\n")
            print(cmd)
    
    print(f"\nFinished upload to {ee_asset_path}")

## add_ee_layer

In [19]:
# Import libraries.
import ee
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Processing

Data processing organised into sections.

In [ ]:
# Set working directories
ds_dir = "./dataset"

In [ ]:
# Make directory structure
mkdirs([ds_dir], exist_ok=True)
!ls

dataset


## Get data package

+ data delivered to GCS bucket, as many small images
+ combine into single GeoTIFF
+ upload to GCS

In [ ]:
!gdalinfo --version

GDAL 2.2.3, released 2017/11/20


### Convert to single tiles

In [ ]:
%%time
for y in [1996, 2007, 2008, 2009, 2010, 2015, 2016]:
  print("\nCopying files to VM\n")
  copy_gcs([f"{gcs_prefix}/land-cover/gmw{y}v2.0"], [ds_dir])
  print("\nBuilding VRT\n")
  !gdalbuildvrt gmw{y}v2_0.vrt {ds_dir}/gmw{y}v2.0/*.tif
  print("\nBuilding GeoTIFF\n")
  !gdal_translate -co "BLOCKXSIZE=512" -co "BLOCKYSIZE=512" -co "TILED=YES" -co "BIGTIFF=YES" -co "COMPRESS=DEFLATE" gmw{y}v2_0.vrt gmw{y}v2_0.tif
  print("\nCopying files to GCS\n")
  copy_gcs([f"./gmw{y}v2_0.tif"], [f"{gcs_prefix}/land-cover"])


Copying files to VM

Processing: gsutil -m cp -r  gs://mangrove_atlas/land-cover/gmw1996v2.0 ./dataset
Task created
Finished copy

Building VRT

0...10...20...30...40...50...60...70...80...90...100 - done.

Building GeoTIFF

Input file size is 1620001, 328501
0...10...20...30...40...50...60...70...80...90...100 - done.

Copying files to GCS

Processing: gsutil -m cp -r  gmw1996v2_0.tif gs://mangrove_atlas/land-cover
Task failed
Finished copy

Copying files to VM

Processing: gsutil -m cp -r  gs://mangrove_atlas/land-cover/gmw2007v2.0 ./dataset
Task created
Finished copy

Building VRT

0...10...20...30...40...50...60...70...80...90...100 - done.

Building GeoTIFF

Input file size is 1620001, 328501
0...10...20...30...40...50...60...70...80...90...100 - done.

Copying files to GCS

Processing: gsutil -m cp -r  gmw2007v2_0.tif gs://mangrove_atlas/land-cover
Task failed
Finished copy

Copying files to VM

Processing: gsutil -m cp -r  gs://mangrove_atlas/land-cover/gmw2008v2.0 ./dataset
Ta

In [ ]:
copy_gcs(["./*.tif"], [f"{gcs_prefix}/land-cover"])

Processing: gsutil -m cp -r  ./*.tif gs://mangrove_atlas/land-cover
Task created
Finished copy


## Upload to earthengine

The dataset is multiple single images (on GCS), each with different timestamp.

+ Upload each image to imageCollection
+ Export as single image
+ Update image metadata
+ Delete tmp files and imageCollection

In [ ]:
# Authenticate earthengine
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=SYiQv7dtMUJQQ429gVxHQM1xxEDEd72tZ9eL0NmX81c&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/0wE2GwOLFvqynOnem34TO4-vlKysb0ql27K9txeVByYsTyhio6K38Mw

Successfully saved authorization token.


In [ ]:
#!earthengine upload image -h

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=KXtTKUpHVxk5pdCsDOGKbSbV0u3DBIPyRaGPP6VgEHc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/0wExYGXXeB8VJDOfffcN6XEGnT5aK95riCFyqNHlzdHxH98pq8Y1qoo

Successfully saved authorization token.


### Add to imageCollection

In [ ]:
# Define parameters

# asset path
ee_asset_path = f"projects/global-mangrove-watch/land-cover/mangrove-extent_version-2-0_1996--2016"

# get description from file
copy_gcs(["gs://mangrove_atlas/land-cover/mangrove-extent_version-2-0_1996--2016_description.md"], ["."])
with open("mangrove-extent_version-2-0_1996--2016_description.md", "r") as f:
  description = f.read()

# set collection properties (these are compatible with Skydipper.Dataset.Metadata)
collection_properties = {
    'name': "Global extent of mangrove forests",
    'version': "2.0",
    'creator': "Global Mangrove Watch (GMW): Aberystwyth University/soloEO/Wetlands International/UNEP-WCMC/JAXA/DOB Ecology",
    'description': description,
    'identifier': "",
    'keywords': "Erosion; Coasts; Natural Infrastructure; Biodiversity; Blue Carbon; Forests; Mangroves; Landcover",
    'citation': "Bunting, Pete, Ake Rosenqvist, Richard M. Lucas, Lisa-Maria Rebelo, Lammert Hilarides, Nathan Thomas, Andy Hardy, Takuya Itoh, Masanobu Shimada, and C. Max Finlayson. “The Global Mangrove Watch—A New 2010 Global Baseline of Mangrove Extent.” Remote Sensing 10, no. 10 (October 2018): 1669. doi: 10.3390/rs10101669.",
    'license': "https://creativecommons.org/licenses/by/4.0/",
    'url': "",
    'language': 'en', 
    'altName': "Global Mangrove Watch, Version 2.0",
    'distribution': "",
    'variableMeasured': "Presence of mangrove forest habitat",
    'units': "1",
    'spatialCoverage': "Global tropics",
    'temporalCoverage': "1996--2016",
    'dataLineage': "Raster data supplied by Aberystwyth University (Dr. Dave Bunting) as tilesets per year, each tilset was combined, and added to Google earth engine as multi-temporal ImageCollection."
}

# set individual image properties (minimal)
image_properties = {
    "band_nodata_values": 0,
    "band_pyramiding_policies": "mode",
    "band_names": "lc"
}

Processing: gsutil -m cp -r  gs://mangrove_atlas/land-cover/mangrove-extent_version-2-0_1996--2016_description.md .
Task created
Finished copy


In [ ]:
# Create ee.ImageCollection, add collecion_properties, and create upload tasks
# You can use dry_run = True to check cmds before running process!
gee_upload_images_to_collection(
    uri_prefix = gcs_prefix,
    dir_path = "land-cover",
    file_pattern="*.tif",
    ee_asset_path = ee_asset_path,
    image_start_times = [f"{y}-01-01" for y in [1996, 2007, 2008, 2009, 2010, 2015, 2016]],
    band_names = ["lc"],
    band_pyramiding_policys = ["mode"],
    band_nodata_values = [0],
    collection_properties = collection_properties,
    image_properties=[image_properties],
    gcs_tmp_path = None,
    force=True,
    dry_run=False
    )
  

Processing: gsutil -m cp -r  gs://mangrove_atlas/land-cover/mangrove-extent_version-2-0_1996--2016_description.md .
Task created
Finished copy

Getting file paths...

Searching gs://mangrove_atlas/land-cover/*.tif


Found 7 path(s)




Checking if collection exists...

ee.ImageCollection projects/global-mangrove-watch/land-cover/mangrove-extent_version-2-0_1996--2016 exists, with properties
:
{
  "bands": [],
  "id": "projects/global-mangrove-watch/land-cover/mangrove-extent_version-2-0_1996--2016",
  "properties": {
    "altName": "Global Mangrove Watch, Version 2.0",
    "citation": "Bunting, Pete, Ake Rosenqvist, Richard M. Lucas, Lisa-Maria Rebelo, Lammert Hilarides, Nathan Thomas, Andy Hardy, Takuya Itoh, Masanobu Shimada, and C. Max Finlayson. \\u201cThe Global Mangrove Watch\\u2014A New 2010 Global Baseline of Mangrove Extent.\\u201d Remote Sensing 10, no. 10 (October 2018): 1669. doi: 10.3390/rs10101669.",
    "creator": "Global Mangrove Watch (GMW): Aberystwyth University/solo



Finished upload to projects/global-mangrove-watch/land-cover/mangrove-extent_version-2-0_1996--2016


In [ ]:
# Check task status
!earthengine --no-use_cloud_api task list --status RUNNING

3KTHJFEL6KBRVPOHQOG2R5YE  Upload        Asset ingestion: projects/global-mangrov..  RUNNING    ---
EEEFJ3ZFFRLWKO6XZDPDIXJS  Upload        Asset ingestion: projects/global-mangrov..  RUNNING    ---
PLXDVQIO3SSHUD7UHHZGSI5C  Upload        Asset ingestion: projects/global-mangrov..  RUNNING    ---
4GAVASGWWSZAVFNZZHJYRJVO  Upload        Asset ingestion: projects/global-mangrov..  RUNNING    ---
UBVESB4EQRV3YXB7QX5EYALL  Upload        Asset ingestion: projects/global-mangrov..  RUNNING    ---
KPZVK7EMSU3VKLTHCUNXGIHW  Upload        Asset ingestion: projects/global-mangrov..  RUNNING    ---
QBTTVD63RCW34WUF4AWNI3RS  Upload        Asset ingestion: projects/global-mangrov..  RUNNING    ---


In [ ]:
# Check ImageCollection properties
!earthengine --no-use_cloud_api asset info {ee_asset_path}

Instructions for updating:
non-resource variables are not supported in the long term
{
  "bands": [],
  "id": "projects/global-mangrove-watch/land-cover/mangrove-extent_version-2-0_1996--2016",
  "properties": {
    "altName": "Global Mangrove Watch, Version 2.0",
    "citation": "Bunting, Pete, Ake Rosenqvist, Richard M. Lucas, Lisa-Maria Rebelo, Lammert Hilarides, Nathan Thomas, Andy Hardy, Takuya Itoh, Masanobu Shimada, and C. Max Finlayson. \\u201cThe Global Mangrove Watch\\u2014A New 2010 Global Baseline of Mangrove Extent.\\u201d Remote Sensing 10, no. 10 (October 2018): 1669. doi: 10.3390/rs10101669.",
    "creator": "Global Mangrove Watch (GMW): Aberystwyth University/soloEO/Wetlands International/UNEP-WCMC/JAXA/DOB Ecology",
    "dataLineage": "Raster data supplied by Aberystwyth University (Dr. Dave Bunting) as tilesets per year, each tilset was combined, and added to Google earth engine as multi-temporal ImageCollection.",
    "description": "# Global extent of mangrove fore

## View

In [ ]:
# Import libraries.
import ee
import folium

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Fetch ImageCollection.
ic = ee.ImageCollection(ee_asset_path)

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 1,
  'palette': ['006633', '#088A08']}

# Create a folium map object.
my_map = folium.Map(location=[-7.998, 39.4767], zoom_start=9, height=500)

for y in [1996, 2007, 2008, 2009, 2010, 2015, 2016]:
  im = ic.filterDate(f"{y}-01-01")
  # Add the image to the map object.
  my_map.add_ee_layer(im.updateMask(im.gt(0)), vis_params, f"Mangrove presence {y}")

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

## Create distance from mangrove habitat image collection

In [ ]:
# Authenticate earthengine
!earthengine authenticate

### Add to imageCollection

In [51]:
# Define parameters

# asset path
ee_asset_path = f"projects/global-mangrove-watch/land-cover/mangrove-extent-distance_version-2-0_1996--2016"

# get description from file
copy_gcs(["gs://mangrove_atlas/land-cover/mangrove-extent-distance_version-2-0_1996--2016_description.md"], ["."])
with open("mangrove-extent-distance_version-2-0_1996--2016_description.md", "r") as f:
  description = f.read()

# set collection properties (these are compatible with Skydipper.Dataset.Metadata)
collection_properties = {
    'name': "Global distance from mangrove forests",
    'version': "2.0",
    'creator': "Global Mangrove Watch (GMW): Aberystwyth University/soloEO/Wetlands International/UNEP-WCMC/JAXA/DOB Ecology, vizzuality.",
    'description': description,
    'identifier': "",
    'keywords': "Erosion; Coasts; Natural Infrastructure; Biodiversity; Blue Carbon; Forests; Mangroves; Landcover; Distance",
    'citation': "Bunting, Pete, Ake Rosenqvist, Richard M. Lucas, Lisa-Maria Rebelo, Lammert Hilarides, Nathan Thomas, Andy Hardy, Takuya Itoh, Masanobu Shimada, and C. Max Finlayson. “The Global Mangrove Watch—A New 2010 Global Baseline of Mangrove Extent.” Remote Sensing 10, no. 10 (October 2018): 1669. doi: 10.3390/rs10101669.",
    'license': "https://creativecommons.org/licenses/by/4.0/",
    'url': "",
    'language': 'en', 
    'altName': "Global Mangrove Watch distance, Version 2.0",
    'distribution': "",
    'variableMeasured': "Linear distance from mangrove forest habitat",
    'units': "m",
    'spatialCoverage': "Global tropics",
    'temporalCoverage': "1996--2016",
    'dataLineage': "Raster data supplied by Aberystwyth University (Dr. Dave Bunting) as tilesets per year, each tilset was combined, and added to Google earth engine as multi-temporal ImageCollection. Squared euclidean distance from pixels with mangrove presence was calculated in earthengine using fastDistance transform, and converted to linear distance in meters using the square-root of pixel area."
}

# set individual image properties (minimal)
image_properties = {
    "band_nodata_values": 'nan',
    "band_pyramiding_policies": "mean",
    "band_names": "distance_m"
}

Processing: gsutil -m cp -r  gs://mangrove_atlas/land-cover/mangrove-extent-distance_version-2-0_1996--2016_description.md .
Task failed
Finished copy


FileNotFoundError: ignored

In [ ]:
# Create image collection
gee_create_image_collection(ee_asset_path, image_start_times, collection_properties = {}, dry_run=Rtue)

In [20]:
# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Zea2s9SuKkqY9JeVb_k3ccyR7SmGC5O4JSJ7ISZPhfM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1gFxGfD3NN-1AZoHdqlfOhDNATHM1UDy6Z1DKc_DwNMLXWZm0FlQaZ4

Successfully saved authorization token.


In [21]:
# Initialize the library.
ee.Initialize()

In [48]:
# Create rasters representing the linear distance from mangrove habitat for each period

# Import libraries.
import ee
import folium
import json
import pprint

def app(year_list, debug):

  # GET DATA LAYERS
  land_cover = ee.ImageCollection('projects/global-mangrove-watch/land-cover/mangrove-extent_version-2-0_1996--2016')

  # SET EXPORT REGION
  region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], 'EPSG:4326')

  # CALCULATIONS
  def getDistance(im, maxDistance):
    
    dist_pixels = im \
    .fastDistanceTransform(maxDistance, 'pixels', 'squared_euclidean') \
    .sqrt()
    
    pixel_size_m = ee.Image.pixelArea().sqrt()
    return dist_pixels.multiply(pixel_size_m) 
  
  def calc_dist(year, debug):
    
    # time_start
    ts = ee.Date.fromYMD(year, 1, 1)
    # get image
    im = ee.Image(land_cover.filterDate(ts).first()).selfMask()
    # calculate distance
    dist = getDistance(im, 256) \
    .rename('distance_m') \
    .set({
      'system:time_start': ts,
      'units': 'meters'  
    })
    
    # Export params
    year_string = ee.Number(year).format()
    nm = ee.String("gl_").cat(year_string).cat("_").cat('distance')
    ns = im.projection().nominalScale()
    
    # EXPORT TO IMAGE COLLECTION
    params = {
        'image': dist,
        'description': "export_" + nm.getInfo(),
        'assetId': 'projects/global-mangrove-watch/land-cover/mangrove_extent_version_2-0_distance_1996-2016/' + nm.getInfo(),
        'pyramidingPolicy':{'distance_m':'mean'},
        'scale': ns.getInfo(),
        'crs': 'EPSG:4326',
        'region': region,
        'maxPixels': 1e13
      
    }
    task = ee.batch.Export.image.toAsset(**params)
    if debug == False:
        task.start()
    
    if debug == True:
      print('\n#######')
      print('\nName:', ee.String(nm).getInfo())
      print('\nNominal scale:', ns.getInfo())
      print('\nDistance image:', json.dumps(dist.getInfo(), indent=4))
      print("\nExport params:\n")
      pprint.pprint(params, indent=4)
      
      print("\n Map:\n")
      # Create a folium map object.
      my_map = folium.Map(location=[-7.998, 39.4767], zoom_start=9, height=500)
      # Set visualization parameters.
      vis_params = {'min': 0, 'max': 50000, 'palette': ['006633', '#088A08']}
      # Add map layer
      my_map.add_ee_layer(dist.updateMask(dist.lte(50000)), vis_params, f"Mangrove distance {year}")
      # Add a layer control panel to the map.
      my_map.add_child(folium.LayerControl())
      # Display the map.
      display(my_map)  
    
    return 'true'
  
  # Loop through year list
  for year in year_list:
    calc_dist(year, debug)
    

# Run process
debug = True
year_list = [1996, 2007, 2008, 2009, 2010, 2015, 2016]
app(year_list, debug)  



#######

Name: gl_1996_distance

Nominal scale: 24.73766462070385

Distance image: {
    "type": "Image",
    "bands": [
        {
            "id": "distance_m",
            "data_type": {
                "type": "PixelType",
                "precision": "double"
            },
            "crs": "EPSG:4326",
            "crs_transform": [
                0.00022222222222201014,
                0,
                -180,
                0,
                -0.00022222222222201014,
                34
            ]
        }
    ],
    "properties": {
        "system:time_start": {
            "type": "Date",
            "value": 820454400000
        },
        "units": "meters"
    }
}

Export params:

{   'assetId': 'projects/global-mangrove-watch/land-cover/mangrove_extent_version_2-0_distance_1996-2016/gl_1996_distance',
    'crs': 'EPSG:4326',
    'description': 'export_gl_1996_distance',
    'image': <ee.image.Image object at 0x7f1811d85c18>,
    'maxPixels': 10000000000000.0,
    '


#######

Name: gl_2007_distance

Nominal scale: 24.73766462070385

Distance image: {
    "type": "Image",
    "bands": [
        {
            "id": "distance_m",
            "data_type": {
                "type": "PixelType",
                "precision": "double"
            },
            "crs": "EPSG:4326",
            "crs_transform": [
                0.00022222222222201014,
                0,
                -180,
                0,
                -0.00022222222222201014,
                34
            ]
        }
    ],
    "properties": {
        "system:time_start": {
            "type": "Date",
            "value": 1167609600000
        },
        "units": "meters"
    }
}

Export params:

{   'assetId': 'projects/global-mangrove-watch/land-cover/mangrove_extent_version_2-0_distance_1996-2016/gl_2007_distance',
    'crs': 'EPSG:4326',
    'description': 'export_gl_2007_distance',
    'image': <ee.image.Image object at 0x7f1811dd7518>,
    'maxPixels': 10000000000000.0,
    


#######

Name: gl_2008_distance

Nominal scale: 24.73766462070385

Distance image: {
    "type": "Image",
    "bands": [
        {
            "id": "distance_m",
            "data_type": {
                "type": "PixelType",
                "precision": "double"
            },
            "crs": "EPSG:4326",
            "crs_transform": [
                0.00022222222222201014,
                0,
                -180,
                0,
                -0.00022222222222201014,
                34
            ]
        }
    ],
    "properties": {
        "system:time_start": {
            "type": "Date",
            "value": 1199145600000
        },
        "units": "meters"
    }
}

Export params:

{   'assetId': 'projects/global-mangrove-watch/land-cover/mangrove_extent_version_2-0_distance_1996-2016/gl_2008_distance',
    'crs': 'EPSG:4326',
    'description': 'export_gl_2008_distance',
    'image': <ee.image.Image object at 0x7f1811dd7828>,
    'maxPixels': 10000000000000.0,
    


#######

Name: gl_2009_distance

Nominal scale: 24.73766462070385

Distance image: {
    "type": "Image",
    "bands": [
        {
            "id": "distance_m",
            "data_type": {
                "type": "PixelType",
                "precision": "double"
            },
            "crs": "EPSG:4326",
            "crs_transform": [
                0.00022222222222201014,
                0,
                -180,
                0,
                -0.00022222222222201014,
                34
            ]
        }
    ],
    "properties": {
        "system:time_start": {
            "type": "Date",
            "value": 1230768000000
        },
        "units": "meters"
    }
}

Export params:

{   'assetId': 'projects/global-mangrove-watch/land-cover/mangrove_extent_version_2-0_distance_1996-2016/gl_2009_distance',
    'crs': 'EPSG:4326',
    'description': 'export_gl_2009_distance',
    'image': <ee.image.Image object at 0x7f1811d70908>,
    'maxPixels': 10000000000000.0,
    


#######

Name: gl_2010_distance

Nominal scale: 24.73766462070385

Distance image: {
    "type": "Image",
    "bands": [
        {
            "id": "distance_m",
            "data_type": {
                "type": "PixelType",
                "precision": "double"
            },
            "crs": "EPSG:4326",
            "crs_transform": [
                0.00022222222222201014,
                0,
                -180,
                0,
                -0.00022222222222201014,
                34
            ]
        }
    ],
    "properties": {
        "system:time_start": {
            "type": "Date",
            "value": 1262304000000
        },
        "units": "meters"
    }
}

Export params:

{   'assetId': 'projects/global-mangrove-watch/land-cover/mangrove_extent_version_2-0_distance_1996-2016/gl_2010_distance',
    'crs': 'EPSG:4326',
    'description': 'export_gl_2010_distance',
    'image': <ee.image.Image object at 0x7f1811d70908>,
    'maxPixels': 10000000000000.0,
    


#######

Name: gl_2015_distance

Nominal scale: 24.73766462070385

Distance image: {
    "type": "Image",
    "bands": [
        {
            "id": "distance_m",
            "data_type": {
                "type": "PixelType",
                "precision": "double"
            },
            "crs": "EPSG:4326",
            "crs_transform": [
                0.00022222222222201014,
                0,
                -180,
                0,
                -0.00022222222222201014,
                34
            ]
        }
    ],
    "properties": {
        "system:time_start": {
            "type": "Date",
            "value": 1420070400000
        },
        "units": "meters"
    }
}

Export params:

{   'assetId': 'projects/global-mangrove-watch/land-cover/mangrove_extent_version_2-0_distance_1996-2016/gl_2015_distance',
    'crs': 'EPSG:4326',
    'description': 'export_gl_2015_distance',
    'image': <ee.image.Image object at 0x7f1811dd7550>,
    'maxPixels': 10000000000000.0,
    


#######

Name: gl_2016_distance

Nominal scale: 24.73766462070385

Distance image: {
    "type": "Image",
    "bands": [
        {
            "id": "distance_m",
            "data_type": {
                "type": "PixelType",
                "precision": "double"
            },
            "crs": "EPSG:4326",
            "crs_transform": [
                0.00022222222222201014,
                0,
                -180,
                0,
                -0.00022222222222201014,
                34
            ]
        }
    ],
    "properties": {
        "system:time_start": {
            "type": "Date",
            "value": 1451606400000
        },
        "units": "meters"
    }
}

Export params:

{   'assetId': 'projects/global-mangrove-watch/land-cover/mangrove_extent_version_2-0_distance_1996-2016/gl_2016_distance',
    'crs': 'EPSG:4326',
    'description': 'export_gl_2016_distance',
    'image': <ee.image.Image object at 0x7f1811dd79e8>,
    'maxPixels': 10000000000000.0,
    

## Create API DataSet, Layers, and Metadata

In [50]:
# Define parameters

# asset path
ee_asset_path = f"projects/global-mangrove-watch/land-cover/mangrove-extent_version-2-0_1996--2016"


# get description
copy_gcs(["gs://mangrove_atlas/land-cover/mangrove-extent_version-2-0_1996--2016_description.md"], ["."])
with open("mangrove-extent_version-2-0_1996--2016_description.md", "r") as f:
  description = f.read()

# set collection properties (these are compatible with Skydipper.Dataset.Metadata)
collection_properties = {
    'name': "Global extent of mangrove forests",
    'version': "2.0",
    'creator': "Global Mangrove Watch (GMW): Aberystwyth University/soloEO/Wetlands International/UNEP-WCMC/JAXA/DOB Ecology",
    'description': description,
    'identifier': "",
    'keywords': "Erosion; Coasts; Natural Infrastructure; Biodiversity; Blue Carbon; Forests; Mangroves; Landcover",
    'citation': "Bunting, Pete, Ake Rosenqvist, Richard M. Lucas, Lisa-Maria Rebelo, Lammert Hilarides, Nathan Thomas, Andy Hardy, Takuya Itoh, Masanobu Shimada, and C. Max Finlayson. “The Global Mangrove Watch—A New 2010 Global Baseline of Mangrove Extent.” Remote Sensing 10, no. 10 (October 2018): 1669. doi: 10.3390/rs10101669.",
    'license': "https://creativecommons.org/licenses/by/4.0/",
    'url': "",
    'language': 'en', 
    'altName': "Global Mangrove Watch, Version 2.0",
    'distribution': "",
    'variableMeasured': "Presence of mangrove forest habitat",
    'units': "1",
    'spatialCoverage': "Global tropics",
    'temporalCoverage': "1996--2016",
    'dataLineage': "Raster data supplied by Aberystwyth University (Dr. Dave Bunting) as tilesets per year, each tilset was combined, and added to Google earth engine as multi-temporal ImageCollection."
}

# set individual image properties (minimal)
image_properties = {
    "band_nodata_values": 0,
    "band_pyramiding_policies": "mode",
    "band_names": "lc"
}



Processing: gsutil -m cp -r  gs://mangrove_atlas/land-cover/mangrove-extent_version-2-0_1996--2016_description.md .
Task created
Finished copy


In [ ]:
import Skydipper as sky

In [ ]:
sky_application = ['mangroveAtlas']

### Create Sky API dataset

In [ ]:
atts = { 
    'name': collection_properties.get("name"),
    'application': sky_application,
    'connectorType': 'rest',
    'provider': 'gee',
    'tableName': ee_asset_path,
    'env': 'production'
}
#print(atts)
ds = sky.Dataset(attributes=atts)
print(ds)


Please enter your the email address associated with your Skydipper accountedward.morris@vizzuality.com
Please enter your Skydipper passwordGjG;g9TaX4k61=!&qVQ2!_ihU3
Dataset dcbfe233-d24c-4c86-8027-86575d2ecb12 Global extent of mangrove forests


### Create Dataset Metadata

In [ ]:
ds = sky.Dataset("dcbfe233-d24c-4c86-8027-86575d2ecb12")
ds

Dataset dcbfe233-d24c-4c86-8027-86575d2ecb12 Global extent of mangrove forests

In [ ]:
sky.Metadata()

In [ ]:
import requests
import json


dataset_id = "dcbfe233-d24c-4c86-8027-86575d2ecb12"
payload = {
"application": ["mangroveAtlas"],
"language": "en",
"name": "global-extent-of-mangrove-forests-version-2-0",
"altName": "<String>",
"description": "<String>, required",
"keywords": "<Array>, required",
"creator": "<String>",
"license": "<String>",
"source": "<String>",
"url": "<String>",
"citation": "<String>",
"distribution": "<Object>",
"identifier": "<String>",
"variableMeasured": "<Object> or <String>",
"units": "<Object>",
"version": "<String>",
"spatialCoverage": "<Object> or <String>",
"temporalCoverage": "<Object> or <String>",
"dataLineage": "<Object> or <String>",
"info": "<Object>",
"fields": "<Object>",
"status": "published"
}

payload = {
   "application": "mangroveAtlas",
   "language": "en"
}
payload = json.dumps(payload)
print(payload)
url = f'{base_url}/v1/dataset/{dataset_id}/metadata'
r = requests.post(url, json=payload, headers=headers)
print(r.url)
print(r.status_code)
print(r.json())

{"application": "mangroveAtlas", "language": "en"}
https://api.skydipper.com/v1/dataset/dcbfe233-d24c-4c86-8027-86575d2ecb12/metadata
400


JSONDecodeError: ignored

### Create Dataset Layers

## Clean up

+ remove data from tmp dir
+ remove ee.Image.collection

In [ ]:
#!gsutil -m rm -r -q {gcs_tmp_path}/*.tif

In [ ]:
#!earthengine --no-use_cloud_api rm -r {asset_path}